<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/tp2_p2_MPI/TP_2_Parte2_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programación Concurrente - TP2 - Parte 2

Para este ejercicio se ha optado por aplicar el tema teórico **MPI** (Message Passing Interface). La finalidad del ejercicio es ampliar el conocimiento sobras la manera que posee Python para implementar la comunicación entre distintos procesos con el uso de una librería denominada **MPI4py** [1]. Para más información puede consultar la última revisión en [2].

---
## 2.1. Ejercicio - Hola Mundo con MPI

### 2.1.1. Preguntas del ejercicio

Ejecute este ejemplo para $4$ o más instancias y responda:

a) ¿Qué función realiza la instancia maestra? ¿Qué función realizan las instancias esclavas?

La instancia maestra en el programa crea aleatoriamente tiempo a consumir en un vector de enteros. Una vez creado los tiempos distribuye esos valores a las distintas esclavas. Las instancias esclavas toman los valores procesandolo en pantalla y una vez terminado dan aviso que culminaron. Los procesos maestros van derivando mas tarea a medida que los esclavos van terminando y dando aviso de sus tareas a los maestros.
Una vez terminadas todas las tareas, se cierra el ciclo de ejecución del programa.

b) ¿Cuántas de esas instancias ejecuta la función `main()`, `initWork()` y `doWork()`?

En main() se estaría conteniendo a todas las instancias, tanto el maestro como los esclavos. initWork() es en donde la funcion maestra distribuye las tareas y solo se estaría ejecutando una instancia.
Por el contrario, en doWork se ejecutan todas las instancias esclavas que van recibiendo los parámetros y van procesando la información.

c) ¿Cómo se diferencian los mensajes de trabajo o de finalización?

La instancia maestra informa por medio de tag=WORK_FLAG a los procesos esclavos para indicar que tiene pendiente horas a procesar. Una vez liquidado todas las posiciones del array provistos para las instancias, se envía un parametro tag=END_WORK_FLAG, que indica que a culminado el procesamiento de todas las posiciones por parte del proceso maestro.

d) ¿Cómo implementaría la función BLAS `axpy()` con este patrón?¿Sería eficiente? *Tips*: Pide solo el planteo, no la implementación.

Se debería correr a principio del programa la seccion secuencial y luego con -np $NRO realizar la parte de paralelismo con n procesos y ver los resultados de cada caso. La eficiencia dependerá de la cantidad de procesos y el hardware disponible para los procesos.

e) ¿Qué sucede cuando solo ejecuta con una sola instancia?

No se ejecuta ya que hay una restricción (numProcesses-1)*4 que determina que si no se puede definir esclavos, la instancia del programa no se ejecute.

f) *Punto opcional*: El código que ejecutan las instancias esclavas, tienen un error en su lógica. ¿Cómo se podría solucionar?

###2.1.2. Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [1]:
! pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746523 sha256=624a2a639c3b35c743e63b4cab41318274cd2b563348efe981132d31e49182da
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


### 2.1.3. Código del programa en Lenguaje Python

In [ ]:
%%writefile Ejercicio2.py
from mpi4py import MPI
import numpy as np
import time

# --------------------------------------------
# Formulario
Max_tiempo_sleep =   1#@param {type: "slider", min: 1, max: 10}
Min_tiempo_sleep =   0#@param {type: "slider", min: 0, max: 10}
# --------------------------------------------

# --------------------------------------------
# Constantes de comunicacion
WORK_FLAG = 1
END_WORK_FLAG = 2
# --------------------------------------------

def main():
    comm = MPI.COMM_WORLD # Instanciamos el tipo de comunicador a utilizar.
    id = comm.Get_rank()  # Obtenemos el id como atributo del proceso que se ejecuta.

    # Utilizamos el 0 para definir al procesos Maestro, cualquier otro id sera un esclavo.
    if (id == 0) :
        init() # Llamamos funcion init para eventos que requeriremos inicialmente solo 1 vez.
        numProcesses = comm.Get_size()  # Obtenemos el numero de procesos totales ejecutados.
        numTasks = (numProcesses-1)*4 # Se setea el numero de tareas.
        workTimes = generateTasks(numTasks) # Se generan las tareas, en este caso seran
        print("El jefe crea {} horas de descanso de sus empleados:".format(workTimes.size), flush=True)
        print(workTimes, flush=True)
        initWork(comm, workTimes, numProcesses)
    else:
        doWork(comm)

def generateTasks(numTasks):
    #TODO: Cambiar la semilla del random para que se generen efectivamente diferentes numeros.
    np.random.seed(1000)
    return np.random.randint(low=Min_tiempo_sleep, high=Max_tiempo_sleep, size=numTasks)

def init():
  print()
  print("Version MPI4py utilizada: {}".format(MPI.Get_version()), flush=True)
  print()
  print( "------------------------------------", flush=True)
  print( "Sistema de trabajo Suizo:", flush=True)
  print( "------------------------------------", flush=True)
  print()

def initWork(comm, workTimes, numProcesses):
    totalWork = workTimes.size
    workcount = 0
    recvcount = 0

    print("Jefe enviando las tareas iniciales:", flush=True)
    for id in range(1, numProcesses):
        if workcount < totalWork:
            work=workTimes[workcount]
            comm.send(work, dest=id, tag=WORK_FLAG) # Envia mensaje de iniciar trabajo con el dato correspondiente del array.
            workcount += 1
            print("Jefe envia trabajo y {} hs de descanso al empleado {}.".format(work, id), flush=True)
    print( "------------------------------------", flush=True)

    # Mientras haya trabajo, se recibe el resultado de los empleados y se sigue enviando MAS trabajo.
    while (workcount < totalWork) :
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat) # Recivimos resultados de los empleados.
        recvcount += 1
        workerId = stat.Get_source() # Obtenemos el identificador del empleado.
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)
        #send next work
        work=workTimes[workcount]
        comm.send(work, dest=workerId, tag=WORK_FLAG)
        workcount += 1
        print("Jefe envia nuevo trabajo y {} hs de descanso al empleado {}.".format(work, workerId), flush=True)

    while (recvcount < totalWork):
        stat = MPI.Status()
        workTime = comm.recv(source=MPI.ANY_SOURCE, status=stat)
        recvcount += 1
        workerId = stat.Get_source()
        print("Jefe recibe trabajo completado {} del empleado {}.".format(workTime, workerId), flush=True)

    for id in range(1, numProcesses):
        comm.send(0, dest=id, tag=END_WORK_FLAG)


def doWork(comm):
    while(True):
        stat = MPI.Status() # Obtiene el estado actual del empleado.
        waitTime = comm.recv(source=0, tag=MPI.ANY_TAG, status=stat) # Obtiene lo enviado por el Jefe.
        print("Soy el empleado con id {}, toca descanzo por {} hs.".format(comm.Get_rank(), waitTime), flush=True)

        if (stat.Get_tag() == END_WORK_FLAG):
            print("Marca tarjeta el empleado {}.".format(comm.Get_rank()), flush=True)
            return
        time.sleep(waitTime)
        comm.send(waitTime, dest=0)

main()


Writing Ejercicio2.py


### 2.1.4 Ejecución del programa

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }
NRO =   1#@param {type: "number"}
# --------------------------------------------

! mpirun --oversubscribe --allow-run-as-root -np $NRO python Ejercicio2.py


Version MPI4py utilizada: (3, 1)

------------------------------------
Sistema de trabajo Suizo:
------------------------------------

El jefe crea 0 horas de descanso de sus empleados:
[]
Jefe enviando las tareas iniciales:
------------------------------------


---
## 3.1 Ejercicio Contar palabras

Desarrollar un programa que permita obtener el valor máximo de un conjunto dado de forma distribuida.

Las condiciones a tener en cuenta son:


*   Debe trabajar con al menos, 4 procesos.
*   El resultado final debe ser informado en cada proceso.
*   Implementar comunicación por Buffer



###3.1.1 Preparación

In [2]:
%%writefile mpi_tp4.py
from mpi4py import MPI
import numpy as np
from IPython.display import display

def main():
  comm = MPI.COMM_WORLD
  numProcesses = comm.Get_size() # numero de procesamientos
  rank = comm.Get_rank()

  count = 5
  sendbuf = None

  if rank == 0:
    print("nro de procesos : {}".format(numProcesses), flush=True)
    sendbuf = np.random.randn(numProcesses, count)
    print()
  else:
    sendbuf = None

  smallerPart = np.empty(count, dtype='float')  # allocate space for result on each process
  comm.Scatter(sendbuf, smallerPart, root=0)
  max = comm.reduce(smallerPart.max(), op=MPI.MAX)

  print("Rank {} has data: {}".format(rank+1, smallerPart), flush=True)

  if rank == 0:
    print("\nMaster {} of {} has original array after Scatter: \n{}\n"\
    .format(rank+1, numProcesses, sendbuf), flush=True)
    data = { 'info': max }
  else:
    data = {}

  #initiate and complete the broadcast
  data = comm.bcast(data, root=0)

  print("Rank {} got max value: {} ".format(rank+1, data['info']), flush=True)

main()

Writing mpi_tp4.py


In [4]:
# --------------------------------------------
# Formulario
NRO =   5#@param {type: "slider", min: 4, max: 10}
# --------------------------------------------

! mpirun --oversubscribe --allow-run-as-root -np $NRO python mpi_tp4.py

nro de procesos : 5

Rank 5 has data: [-0.47864939 -1.38943525 -0.76752844 -1.28487341  0.02915897]
Rank 2 has data: [ 1.42185048  0.28112614 -0.5163519   0.32307783  2.05472753]
Rank 4 has data: [-1.54921043 -0.49624981  1.43420213  0.3276727   0.55060586]
Rank 3 has data: [ 0.48342971 -0.94074541  0.30238038 -2.30662505  1.1107557 ]
Rank 1 has data: [0.95926544 1.91702087 0.3194928  0.90779229 0.87356754]

Master 1 of 5 has original array after Scatter: 
[[ 0.95926544  1.91702087  0.3194928   0.90779229  0.87356754]
 [ 1.42185048  0.28112614 -0.5163519   0.32307783  2.05472753]
 [ 0.48342971 -0.94074541  0.30238038 -2.30662505  1.1107557 ]
 [-1.54921043 -0.49624981  1.43420213  0.3276727   0.55060586]
 [-0.47864939 -1.38943525 -0.76752844 -1.28487341  0.02915897]]

Rank 1 got max value: 2.054727526578737 
Rank 3 got max value: 2.054727526578737 
Rank 2 got max value: 2.054727526578737 
Rank 5 got max value: 2.054727526578737 
Rank 4 got max value: 2.054727526578737 


---
# Bibliografía

[1] MPI4py: https://mpi4py.readthedocs.io/en/stable/

[2] La versión oficial de MPI (Versión 4): https://www.mpi-forum.org/docs/mpi-4.0/mpi40-report.pdf

